In [ ]:
!pip3 install ktrain

In [2]:
import pandas as pd
import numpy as np

#import ktrain
#from ktrain import text

In [7]:
data_train = pd.read_csv('/content/drive/My Drive/IBM_sentiment_analysis/classification_sentiment_analysis/datasets/self-training_iteration2_data/train_set.csv', encoding='utf-8')
data_test = pd.read_csv('/content/drive/My Drive/IBM_sentiment_analysis/classification_sentiment_analysis/datasets/self-training_iteration2_data/test_set.csv', encoding='utf-8')

In [8]:
print(data_train.shape)
print(data_test.shape)
print(data_train.isnull().sum())
print(data_test.isnull().sum())

(19364, 2)
(7222, 2)
Emotion    0
Text       0
dtype: int64
Emotion    0
Text       0
dtype: int64


In [ ]:
data_train.head()

,Emotion,Text
0,fear,It has been a trend for leftists to criticize ...
1,anger,Spitting on people who are saving your lives? ...
2,anger,You complain all the time and then wonder why ...
3,neutral,The #Coronavirus pandemic proves that crises c...
4,fear,And you feel anxious about getting the first l...


In [ ]:
data_test.head()

,Emotion,Text
0,anger,That's really it for my madden career that gam...
1,joy,Heyy congrats yall did it 😤
2,fear,Situation seems to be horrible as per data sho...
3,neutral,Latest on #Coronavirus is that cases have emer...
4,neutral,I've just posted a new blog: Satoshi Nakaboto:...


In [ ]:
X_train = data_train.Text.tolist()
X_test = data_test.Text.tolist()

y_train = data_train.Emotion.tolist()
y_test = data_test.Emotion.tolist()

data = data_train.append(data_test, ignore_index=True)

class_names = ['joy', 'sadness', 'fear', 'anger', 'neutral']

print('size of training set: %s' % (len(data_train['Text'])))
print('size of validation set: %s' % (len(data_test['Text'])))
print(data.Emotion.value_counts())

data.head(10)

size of training set: 19364
size of validation set: 7222
anger      5441
fear       5372
joy        5365
sadness    5313
neutral    5095
Name: Emotion, dtype: int64


,Emotion,Text
0,fear,It has been a trend for leftists to criticize ...
1,anger,Spitting on people who are saving your lives? ...
2,anger,You complain all the time and then wonder why ...
3,neutral,The #Coronavirus pandemic proves that crises c...
4,fear,And you feel anxious about getting the first l...
5,sadness,"I'm down with high fever, terrible throat ache..."
6,sadness,"Requesting Hon'ble garu, garu\nto examine the ..."
7,sadness,Maintain social distancing and stay connected ...
8,joy,` I 'm delighted you 're so pleased to see me ...
9,fear,Everything is okay sister?? Lalu & Dr. khan?? ...


In [ ]:
encoding = {
    'joy': 0,
    'sadness': 1,
    'fear': 2,
    'anger': 3,
    'neutral': 4
}


y_train = [encoding[x] for x in y_train]
y_test = [encoding[x] for x in y_test]

In [ ]:
(x_train,  y_train), (x_test, y_test), preproc = text.texts_from_array(x_train=X_train, y_train=y_train,
                                                                       x_test=X_test, y_test=y_test,
                                                                       class_names=class_names,
                                                                       preprocess_mode='bert',
                                                                       maxlen=350, 
                                                                       max_features=35000)

task: text classification
preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


In [ ]:
model = text.text_classifier('bert', train_data=(x_train, y_train), preproc=preproc)

Is Multi-Label? False
maxlen is 350
done.


In [ ]:
learner = ktrain.get_learner(model, train_data=(x_train, y_train), 
                             val_data=(x_test, y_test),
                             batch_size=6)

In [ ]:
learner.fit_onecycle(2e-5, 3)



begin training using onecycle policy with max lr of 2e-05...
Train on 19364 samples, validate on 7222 samples
Epoch 1/3
19364/19364 [==============================] - 4255s 220ms/sample - loss: 0.4343 - accuracy: 0.8407 - val_loss: 0.2504 - val_accuracy: 0.9191
Epoch 2/3
19364/19364 [==============================] - 4209s 217ms/sample - loss: 0.1380 - accuracy: 0.9543 - val_loss: 0.2587 - val_accuracy: 0.9197
Epoch 3/3
19364/19364 [==============================] - 4241s 219ms/sample - loss: 0.0454 - accuracy: 0.9850 - val_loss: 0.2369 - val_accuracy: 0.9338


In [ ]:
learner.validate(val_data=(x_test, y_test), class_names=class_names)

              precision    recall  f1-score   support

         joy       0.96      0.94      0.95      1450
     sadness       0.92      0.93      0.92      1463
        fear       0.93      0.93      0.93      1469
       anger       0.94      0.92      0.93      1488
     neutral       0.92      0.95      0.94      1352

    accuracy                           0.93      7222
   macro avg       0.93      0.93      0.93      7222
weighted avg       0.93      0.93      0.93      7222



array([[1365,   21,    9,    9,   46],
       [  13, 1361,   33,   34,   22],
       [  17,   38, 1363,   36,   15],
       [  10,   39,   48, 1368,   23],
       [  13,   25,   20,    7, 1287]])

In [ ]:
predictor = ktrain.get_predictor(learner.model, preproc)
predictor.get_classes()

['joy', 'sadness', 'fear', 'anger', 'neutral']

In [ ]:
message = 'hell no'
predictor.predict(message)

'anger'

In [ ]:
predictor.save("/content/drive/My Drive/IBM_sentiment_analysis/classification_sentiment_analysis/models/bert_model_self-training_iteration2")